# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [ ]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [ ]:
df = pd.read_csv('scraped/hm.csv')

In [ ]:
df.head()

In [ ]:



df['price_discounted'] = df.price.str.extract("^[$](\d+[.]?\d+)\s+")
df['price_original'] = df.price.str.extract("\s+[$](\d+[.]?\d+)")



In [ ]:



for index, row in df.iterrows():
    pct_discount = round((1-(df.price_discounted.astype(float))/(df.price_original.astype(float)))*100,2)
    df['pct_discount'] = pct_discount
#df.drop(['pct_discount'],axis = 1)



In [ ]:
df['article_id'] = df.url.str.extract("[?]article=(\d+[-]\w)")

In [ ]:
df.to_csv('hm_cleaned.csv',index=False)

## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [ ]:
df = pd.read_csv("scraped/sci-fi.csv")

In [ ]:
df.title = df.title.str.strip()
df

In [ ]:
# avg_rating
df['avg_rating'] = df.full_rating.str.extract(r"\s(\d[.]\d+)\savg").astype(float)
df['rating_count'] = df.full_rating.str.extract(r"[—]\s(\d+[,]?\d*[,]?\d*)\sratings")


# series the series the book belongs to
# series_no the book in the series that it is
df['total_score'] = df.full_score.str.extract(r"score:\s(\d+[,]\d+)")
df['score_votes'] = df.full_score.str.extract(r"(\d+)\speople")

#I didn't use the comma to end this regex because i found one series without a comma...
df['series'] = df.title.str.extract(r"[(](.*)[#]")

df['series_no'] = df.title.str.extract(r"#(\d+)[)]").astype(float)

df

In [ ]:
df.to_csv('sci-fi_cleaned.csv')

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [ ]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})

In [ ]:
df.to_csv('courts.csv',index=False)

In [ ]:
df = pd.read_csv('courts.csv')
df.head()

Okay, now do the work and **make those new columns!**

In [ ]:
df['lawyer_app'] = df.content.str.extract(r"F\w+ A\w+[:\n]+?(.*\w)[.]?\s")
df

In [ ]:
df['lawyer_gov'] = df.content.str.extract(r"F\w+ G\w+[:\n]+?(.*\w)[,]\s")


In [ ]:
df['judge'] = df.content.str.extract(r"(.*), Administrative Judge[:]")
df

In [ ]:
def get_access(row):
    if re.findall("classified information is granted.",row['content']):
        return "granted"
    elif re.findall("classified information is denied." , row['content']) or re.findall("declined to grant",row['content']):
        return "denied"


In [ ]:
df['access'] = df.apply(get_access, axis = 1)
df

In [ ]:
df.to_csv('court_cleaned.csv')

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [ ]:
contents = open("data/Austen_Pride.txt", encoding="utf8").read()

## Look at the first 500 or so characters of it 

In [ ]:
contents[:500]

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [ ]:

he_she = re.findall(r"\bs?he\b", contents, re.IGNORECASE)
he_she[:100]

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [ ]:
he_she_word = re.findall(r"\bs?he\b \w+",contents, re.IGNORECASE)
he_she_word[:10]

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [ ]:
he_she_word_group = re.findall(r"(\bs?he\b) (\w+)",contents, re.IGNORECASE)
he_she_word_group[:5]

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [ ]:

df = pd.DataFrame(he_she_word_group, columns = ['pronoun', 'verb'])
df

## How many times is each pronoun used?

In [43]:
pd.value_counts(df['pronoun'])



she    1647
he     1288
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [ ]:
df.pronoun.replace('He', "he", inplace=True)
df.pronoun.replace('She','she',inplace = True)
df

## What are the top 20 most common verbs?

In [ ]:
df.verb.value_counts(sort = True).head(20)

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [ ]:
df.groupby("pronoun")['verb'].value_counts().groupby(level=0).head(20)

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [ ]:
df[df['verb'] == 'cried'].pronoun.value_counts(normalize= True)*100

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [39]:
contents2 = open('data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt',encoding="utf8").read()

In [41]:
he_she2 = re.findall(r"\bs?he\b", contents2, re.IGNORECASE)


In [33]:
type(he_she)

list

In [42]:
he_count=0
she_count=0
for item in he_she2:
    if item == 'he' or item == 'He':
        he_count += 1
    elif item == 'she' or item == 'She':
        she_count += 1

        

In [35]:
he_count

3062

In [36]:
she_count

159

In [38]:
diff = he_count - she_count
diff

2903

In [44]:
pd.value_counts(df['pronoun'])

she    1647
he     1288
Name: pronoun, dtype: int64

In [45]:
diff = 1647 - 1288
diff

359